In [1]:
!pwd

/ssl-jet-vol-v3/I-JEPA-Jets/src/dataset


In [2]:
import random
from JetDataset import JetDataset
from torch.utils.data import Dataset, DataLoader

In [3]:
train_dataset = JetDataset("../../data/val/val_20_30_new.h5", num_jets=1000, config=None)

Initializing JetDataset with file: ../../data/val/val_20_30_new.h5
Loading features and subjets from HDF5 file
Filtered to 261239 good jets
Final dataset size: 1000 jets


In [4]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
train_iter = iter(train_loader)
particle_features, subjets, particle_indices, subjet_mask, particle_mask = next(train_iter)

In [5]:
print("particle features", particle_features.shape)
print("subjets", subjets.shape)
print("particle indices", particle_indices.shape)
print("subjet_mask", subjet_mask.shape)
print("particle_mask", particle_mask.shape)

particle features torch.Size([1, 4, 128])
subjets torch.Size([1, 20, 4])
particle indices torch.Size([1, 20, 30])
subjet_mask torch.Size([1, 20])
particle_mask torch.Size([1, 20, 30])


## How to get features of particles contained in a subjet

In [6]:
jet_idx = 0
subjet_idx = random.randint(0, 20)
particle_features_at_subjet_idx = particle_features[0, :, particle_indices[0, subjet_idx].long()]

In [7]:
particle_indices[0, subjet_idx]

tensor([32., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)

In [8]:
particle_features_at_subjet_idx.shape

torch.Size([4, 30])

In [9]:
particle_features_at_subjet_idx

tensor([[-0.5510,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0223,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.6121,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.7517,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000